In [31]:
import numpy as np
import networkx as nx
import random
from gensim.models import Word2Vec

'''
Modified version of code originally by Aditya Grover: https://github.com/aditya-grover/node2vec
'''

class Node2Vec():
    def read_graph(self, filename):
        '''
        Reads the input network in networkx.
        '''
        G = nx.read_edgelist(filename, 
                             nodetype=str, 
                             data=(('weight',float),),
                             delimiter=',',
                             create_using=nx.DiGraph())
       
        return G

    def learn_embeddings(self, walks, dimensions, window, epochs):
        '''
        Learn embeddings by optimizing the Skipgram objective using SGD.
        '''
        walks = [[str(step) for step in walk] for walk in walks]
        self.model = Word2Vec(sentences=walks, 
                              size=dimensions, 
                              window=window, 
                              min_count=0, 
                              sg=1, 
                              workers=7, 
                              iter=epochs)
        return self.model

    def fit_transform(self, edgelist_file, dimensions, window, epochs, num_walks, walk_length, p, q):
        '''
        Pipeline for representational learning for all nodes in a graph.
        '''
        self.G = self.read_graph(edgelist_file)
        self.p = p
        self.q = q
        #print(self.G.graph)
        self.preprocess_transition_probs()
        walks = self.simulate_walks(num_walks, walk_length, p, q)
        return self.learn_embeddings(walks, dimensions, window, epochs=epochs) 

    def node2vec_walk(self, walk_length, start_node):
        '''
        Simulate a random walk starting from start node.
        '''
        G = self.G
        alias_nodes = self.alias_nodes
        alias_edges = self.alias_edges

        walk = [start_node]

        while len(walk) < walk_length:
            cur = walk[-1]
            cur_nbrs = sorted(G.neighbors(cur))
            if len(cur_nbrs) > 0:
                if len(walk) == 1:
                    walk.append(cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
                else:
                    prev = walk[-2]
                    next = cur_nbrs[alias_draw(alias_edges[(prev, cur)][0], 
                        alias_edges[(prev, cur)][1])]
                    walk.append(next)
            else:
                break

        return walk

    def simulate_walks(self, num_walks, walk_length, p, q):
        '''
        Repeatedly simulate random walks from each node.
        '''
        G = self.G
        walks = []
        nodes = list(G.nodes())
        print('Walk iteration: ')
        for walk_iter in range(num_walks):
            print(str(walk_iter+1), '/', str(num_walks))
            random.shuffle(nodes)
            for node in nodes:
                walks.append(self.node2vec_walk(walk_length=walk_length, start_node=node))
            print('num walks ' + str(len(walks)))

        return walks

    def get_alias_edge(self, src, dst):
        '''
        Get the alias edge setup lists for a given edge.
        '''
        G = self.G
        p = self.p
        q = self.q

        unnormalized_probs = []
        for dst_nbr in sorted(G.neighbors(dst)):
            if dst_nbr == src:
                unnormalized_probs.append(G[dst][dst_nbr]['weight']/p)
            elif G.has_edge(dst_nbr, src):
                unnormalized_probs.append(G[dst][dst_nbr]['weight'])
            else:
                unnormalized_probs.append(G[dst][dst_nbr]['weight']/q)
        norm_const = sum(unnormalized_probs)
        normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]

        return alias_setup(normalized_probs)

    def preprocess_transition_probs(self):
        '''
        Preprocessing of transition probabilities for guiding the random walks.
        '''
        G = self.G
        # is_directed = ????

        alias_nodes = {}
        for node in G.nodes():
            unnormalized_probs = [G[node][nbr]['weight'] for nbr in sorted(G.neighbors(node))]
            norm_const = sum(unnormalized_probs)
            normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]
            alias_nodes[node] = alias_setup(normalized_probs)

        alias_edges = {}
        triads = {}

        # TODO I'm assuming the graph is always directed. Do I want to do experiments on undirected graphs?
        #if is_directed:
        for edge in G.edges():
            alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
        #else:
        #    for edge in G.edges():
        #        alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
        #        alias_edges[(edge[1], edge[0])] = self.get_alias_edge(edge[1], edge[0])

        self.alias_nodes = alias_nodes
        self.alias_edges = alias_edges

        return


def alias_setup(probs):
    '''
    Compute utility lists for non-uniform sampling from discrete distributions.
    '''
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)

    smaller = []
    larger = []
    for kk, prob in enumerate(probs):
        q[kk] = K*prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

def alias_draw(J, q):
    '''
    Draw sample from a non-uniform discrete distribution using alias sampling.
    '''
    K = len(J)

    kk = int(np.floor(np.random.rand()*K))
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]

In [33]:
for sample_rate in [.25,.5,.75]:
    sr_str = str(int(sample_rate*100))
    training_edgelist_file = 'data/no_cutoff/train_split_' + sr_str + '.dat'
    # TODO: Determine better values for this
    p_q = [(4,1), (1,4), 
           (3,1), (1,3),
           (2,1), (1,2)]
    for p,q in p_q:
        n2v = Node2Vec()
        model = n2v.fit_transform(training_edgelist_file,
                                  dimensions=50, 
                                  window=5,
                                  epochs=20,
                                  num_walks=10, 
                                  walk_length=25, 
                                  p=p, 
                                  q=q)
        vectorfilename = 'data/vectors/node2vec_sample{}_p{}_q{}.dat'.format(sr_str, p, q)
        vector_file = open(vectorfilename, 'w')
        for i,n in enumerate(n2v.G.nodes):
            vector_file.write(n + ',' + ','.join([str(_) for _ in model.wv.vectors[i,:]]) + '\n')
        vector_file.close()

Walk iteration: 
1 / 10
num walks 14690
2 / 10
num walks 29380
3 / 10
num walks 44070
4 / 10
num walks 58760
5 / 10
num walks 73450
6 / 10
num walks 88140
7 / 10
num walks 102830
8 / 10
num walks 117520
9 / 10
num walks 132210
10 / 10
num walks 146900
Walk iteration: 
1 / 10
num walks 14690
2 / 10
num walks 29380
3 / 10
num walks 44070
4 / 10
num walks 58760
5 / 10
num walks 73450
6 / 10
num walks 88140
7 / 10
num walks 102830
8 / 10
num walks 117520
9 / 10
num walks 132210
10 / 10
num walks 146900
Walk iteration: 
1 / 10
num walks 14690
2 / 10
num walks 29380
3 / 10
num walks 44070
4 / 10
num walks 58760
5 / 10
num walks 73450
6 / 10
num walks 88140
7 / 10
num walks 102830
8 / 10
num walks 117520
9 / 10
num walks 132210
10 / 10
num walks 146900
Walk iteration: 
1 / 10
num walks 14690
2 / 10
num walks 29380
3 / 10
num walks 44070
4 / 10
num walks 58760
5 / 10
num walks 73450
6 / 10
num walks 88140
7 / 10
num walks 102830
8 / 10
num walks 117520
9 / 10
num walks 132210
10 / 10
num walks